In [1]:
# All imports
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import inspect
from tqdm import tqdm
from fastai.imports import *
from fastai.vision.all import *
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.metrics import SparseCategoricalCrossentropy
from tensorflow import keras
from PIL import Image
from numpy import asarray
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow
from scipy.io import loadmat 
import cv2

In [2]:
# Retrieving files from our Google Drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Unzipping files into colab
!unzip '/content/drive/My Drive/NNDL Project Submission/im.zip'

Archive:  /content/drive/My Drive/NNDL Project Submission/im.zip
replace __MACOSX/._im? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
# Input for the labels of our images
labels = '/content/drive/My Drive/NNDL Project/imagelabels.mat'
image_labels = loadmat(labels)
image_labels.keys()

dict_keys(['__header__', '__version__', '__globals__', 'labels'])

In [3]:
# Adding our image_labels into a dataframe to match with our images
df = pd.DataFrame()
labels = image_labels['labels'].transpose()
labels = labels -1
df = pd.DataFrame(labels, columns = ['image_label'])

In [4]:
# Adding our image paths to our image_labels dataframe
image_path = []
for i in range (1,len(labels)+1):
  if i<10:
    x = '0000' + str(i)
  elif i<100:
    x = '000' + str(i)
  elif i<1000:
    x = '00' + str(i)
  elif i<10000:
    x = '0' + str(i)
  else:
    break
  image_path.append('/content/im/image_'+x+'.jpg')

df["image_path"] = image_path 

In [5]:
# Checking how many images we have per class (top few and bottom few to understand the distributioin of samples)
df.image_label.value_counts()

50    258
76    251
45    196
72    194
88    184
     ... 
6      40
44     40
33     40
0      40
26     40
Name: image_label, Length: 102, dtype: int64

In [6]:
# Test to see whether the path code works to open the 1st image
im = Image.open(df['image_path'][0])
width, height = im.size
print(width,height)

591 500


In [7]:
# Final check on our data frame before loading the images and converting them to (size, size, 3) numpy pixels
df

,image_label,image_path
0,76,/content/im/image_00001.jpg
1,76,/content/im/image_00002.jpg
2,76,/content/im/image_00003.jpg
3,76,/content/im/image_00004.jpg
4,76,/content/im/image_00005.jpg
...,...,...
8184,61,/content/im/image_08185.jpg
8185,61,/content/im/image_08186.jpg
8186,61,/content/im/image_08187.jpg
8187,61,/content/im/image_08188.jpg


In [8]:
# Function to load images, convert to numpy array of pixel data, split into train test split, convert to a tensor and return. (Based on img_size input)
def split_function(train_df,img_size):
    data = train_df.copy()
    img_size = img_size
    train_image = []
    for i in tqdm(range(data.shape[0])):
      # inputting images with the required input size
      img = image.load_img(data['image_path'][i],target_size=(img_size,img_size,3))
      img = image.img_to_array(img)
      train_image.append(img)
    # appending the image arrays to a new numpy array
    train_df = np.array(train_image)
    image_label = data['image_label'].to_numpy()

    # Since we want only a small number  of images while training the train val test split has been set accordingly
    # train, val, test split (0.2, 0.4, 0.4)

    X_train, X_test, y_train, y_test = train_test_split(train_df, image_label, random_state=42, test_size=0.75, stratify = image_label)
    X_val, X_train, y_val, y_train = train_test_split(X_train, y_train, random_state=42, test_size=0.75, stratify = y_train)

    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))

    print('Number of training samples ', len(train_ds))
    print('Number of validation samples ', len(val_ds))

    # Batching and applying prefetching for faster processing
    train_ds = train_ds.cache().batch(32).prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
    val_ds = val_ds.cache().batch(32).prefetch(buffer_size = tf.data.experimental.AUTOTUNE)

    print('Number of training samples after batching ', len(train_ds))
    print('Number of validation samples after batching ', len(val_ds))
    
    return(train_ds, val_ds)

In [9]:
# Fetching all pretrained keras models
all_models_dictionary = {m[0]:m[1] for m in inspect.getmembers(tf.keras.applications, inspect.isfunction)}

# List of all pretrained Keras Model
all_models_dictionary

{'DenseNet121': <function keras.applications.densenet.DenseNet121(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000, classifier_activation='softmax')>,
 'DenseNet169': <function keras.applications.densenet.DenseNet169(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000, classifier_activation='softmax')>,
 'DenseNet201': <function keras.applications.densenet.DenseNet201(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000, classifier_activation='softmax')>,
 'EfficientNetB0': <function keras.applications.efficientnet.EfficientNetB0(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000, classifier_activation='softmax', **kwargs)>,
 'EfficientNetB1': <function keras.applications.efficientnet.EfficientNetB1(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None,

In [10]:
# List of all pretrained Keras models specifically for image classification
image_pretrained_models = [
        'VGG16',
        'VGG19',
        'ResNet50',
        'ResNet50V2',
        'ResNet101',
        'ResNet101V2',
        'ResNet152',
        'ResNet152V2',
        'MobileNet',
        'MobileNetV2',
        'DenseNet121',
        'DenseNet169',
        'DenseNet201',
        'NASNetMobile',
        'EfficientNetB0',
        'EfficientNetB1',
        'EfficientNetB2',
        'EfficientNetB3',
        'EfficientNetB4',
        'EfficientNetB5',
        'EfficientNetB6',
        'EfficientNetB7',
        'EfficientNetV2B0',
        'EfficientNetV2B1',
        'EfficientNetV2B2',
        'EfficientNetV2B3',
        'EfficientNetV2S',
        'EfficientNetV2M',
        'EfficientNetV2L',
        'Xception',
        'InceptionV3',
        'InceptionResNetV2',
        'NASNetLarge'
    ]

In [11]:
preprocess_func = { 'VGG16' : tf.keras.applications.vgg16.preprocess_input,
                    'VGG19':tf.keras.applications.vgg19.preprocess_input,
                    'ResNet50':tf.keras.applications.resnet.preprocess_input,
                    'ResNet50V2':tf.keras.applications.resnet_v2.preprocess_input,
                    'ResNet101':tf.keras.applications.resnet.preprocess_input,
                    'ResNet101V2':tf.keras.applications.resnet_v2.preprocess_input,
                    'ResNet152':tf.keras.applications.resnet.preprocess_input,
                    'ResNet152V2':tf.keras.applications.resnet_v2.preprocess_input,
                    'MobileNet':tf.keras.applications.mobilenet.preprocess_input,
                    'MobileNetV2':tf.keras.applications.mobilenet_v2.preprocess_input,
                    'DenseNet121':tf.keras.applications.densenet.preprocess_input,
                    'DenseNet169':tf.keras.applications.densenet.preprocess_input,
                    'DenseNet201':tf.keras.applications.densenet.preprocess_input,
                    'NASNetMobile':tf.keras.applications.nasnet.preprocess_input,
                    'EfficientNetB0':tf.keras.applications.efficientnet.preprocess_input,
                    'EfficientNetB1':tf.keras.applications.efficientnet.preprocess_input,
                    'EfficientNetB2':tf.keras.applications.efficientnet.preprocess_input,
                    'EfficientNetB3':tf.keras.applications.efficientnet.preprocess_input,
                    'EfficientNetB4':tf.keras.applications.efficientnet.preprocess_input,
                    'EfficientNetB5':tf.keras.applications.efficientnet.preprocess_input,
                    'EfficientNetB6':tf.keras.applications.efficientnet.preprocess_input,
                    'EfficientNetB7':tf.keras.applications.efficientnet.preprocess_input,
                    'EfficientNetV2B0':tf.keras.applications.efficientnet_v2.preprocess_input,
                    'EfficientNetV2B1':tf.keras.applications.efficientnet_v2.preprocess_input,
                    'EfficientNetV2B2':tf.keras.applications.efficientnet_v2.preprocess_input,
                    'EfficientNetV2B3':tf.keras.applications.efficientnet_v2.preprocess_input,
                    'EfficientNetV2S':tf.keras.applications.efficientnet_v2.preprocess_input,
                    'EfficientNetV2M':tf.keras.applications.efficientnet_v2.preprocess_input,
                    'EfficientNetV2L':tf.keras.applications.efficientnet_v2.preprocess_input,
                    'Xception': tf.keras.applications.xception.preprocess_input,
                    'InceptionV3':tf.keras.applications.inception_v3.preprocess_input,
                    'InceptionResNetV2':tf.keras.applications.inception_resnet_v2.preprocess_input,
                    'NASNetLarge':tf.keras.applications.nasnet.preprocess_input
                  }

In [12]:
# Fetching only the models specifically mentioned for image classification
image_models = {image_pretrained_model: all_models_dictionary[image_pretrained_model] for image_pretrained_model in image_pretrained_models}

# Corresponds to Keras documentation for image specific pretrained models
len(image_models)

33

In [13]:
# Early stopping implementation to reduce over
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Initialising our augmentation layer
data_augmentation = tf.keras.Sequential([
  keras.layers.RandomFlip("horizontal_and_vertical"),
  keras.layers.RandomRotation(0.2),
])

In [14]:
train_ds_224, val_ds_224 = split_function(df,224)
#train_ds_299, val_ds_299 = split_function(df,299)
#train_ds_331, val_ds_331 = split_function(df,331)

100%|██████████| 8189/8189 [00:46<00:00, 177.14it/s]


Number of training samples  1536
Number of validation samples  511
Number of training samples after batching  48
Number of validation samples after batching  16


In [15]:
# Looping over all available image models in Keras
model_benchmarks = {'pretrained_model_name' : [], 'num_params' : [], 'validation_accuracy' : [] }

train_ds, val_ds = train_ds_224, val_ds_224
cnt = 0

for pretrained_model_name, model in tqdm(image_models.items()):
    if cnt==1:
      break
    # Special handling for different pretrained models since they require different image input sizes
    if pretrained_model_name in ['Xception','InceptionV3','InceptionResNetV2'] :
        input_shape=(299, 299, 3)
        train_ds, val_ds = train_ds_299, val_ds_299

    elif pretrained_model_name == 'NASNetLarge':
        input_shape=(331, 331, 3)
        train_ds, val_ds = train_ds_331, val_ds_331

    else :
        input_shape=(224, 224, 3)
        
    # Creating a new model on top
    inputs = tf.keras.layers.Input(shape=input_shape)
    
    #  Float casting
    x = tf.cast(inputs, tf.float32)

    # Data Augmentation 
    x = data_augmentation(x)

    # Default Preprocess Function for our Pre-trained Models
    x = preprocess_func[pretrained_model_name](x)
    
    # Creating our base model
    base_model = model(weights="imagenet", 
                        include_top=False, 
                        input_shape=input_shape)
    # Frezing the base_model weights
    base_model.trainable = False
     
    # Building the model
    x = base_model(x, training = False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(102, activation='softmax')(x)
    reg_model = tf.keras.Model(inputs, outputs)
    
    reg_model.compile(optimizer=tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    history = reg_model.fit(train_ds, epochs = 20, validation_data=val_ds, callbacks=[es])
    
    # Adding model benchmarks to compare later
    model_benchmarks['pretrained_model_name'].append(pretrained_model_name)
    model_benchmarks['num_params'].append(base_model.count_params())
    model_benchmarks['validation_accuracy'].append(history.history['val_accuracy'][-1])  
    cnt = cnt+1

  0%|          | 0/33 [00:00<?, ?it/s]

58889256/58889256 [==============================] - 3s 0us/step
Epoch 1/20
48/48 [==============================] - 23s 284ms/step - loss: 10.1865 - accuracy: 0.0260 - val_loss: 6.4855 - val_accuracy: 0.1155
Epoch 2/20
48/48 [==============================] - 11s 223ms/step - loss: 6.2390 - accuracy: 0.1315 - val_loss: 4.2173 - val_accuracy: 0.2720
Epoch 3/20
48/48 [==============================] - 11s 227ms/step - loss: 4.1038 - accuracy: 0.2643 - val_loss: 2.9089 - val_accuracy: 0.3875
Epoch 4/20
48/48 [==============================] - 11s 232ms/step - loss: 2.8748 - accuracy: 0.3783 - val_loss: 2.2813 - val_accuracy: 0.4971
Epoch 5/20
48/48 [==============================] - 11s 238ms/step - loss: 2.2354 - accuracy: 0.4733 - val_loss: 1.9543 - val_accuracy: 0.5499
Epoch 6/20
48/48 [==============================] - 11s 239ms/step - loss: 1.8430 - accuracy: 0.5423 - val_loss: 1.6793 - val_accuracy: 0.5871
Epoch 7/20
48/48 [==============================] - 11s 236ms/step - loss: 1

  3%|▎         | 1/33 [04:27<2:22:47, 267.72s/it]


In [16]:
# Converting model benchmarks to a DataFrame

# (Since our colab notebook crashed in the middle we only have the results of the last iteration, the rest of the results are tabulated in the report)

benchmark_df = pd.DataFrame(model_benchmarks)
benchmark_df.sort_values('validation_accuracy', inplace=True, ascending=False)
benchmark_df

,pretrained_model_name,num_params,validation_accuracy
0,VGG16,14714688,0.76908


In [17]:
# Saving the benchmarks as a csv to be used for comparison later
benchmark_df.to_csv("transfer_learning_model_results_low_samples.csv")